In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import common.features_datasets_externos as feature

In [2]:
train = pd.read_csv("sets_de_datos/train.csv", index_col = 0)

In [3]:
test = pd.read_csv("sets_de_datos/test.csv", index_col = 0)

In [4]:
train["fecha"] = pd.to_datetime(train["fecha"], format="%Y-%m-%d %H:%M:%S", errors='coerce')

In [5]:
test["fecha"] = pd.to_datetime(test["fecha"], format="%Y-%m-%d %H:%M:%S", errors='coerce')

## Categorias contempladas hasta el momento:
- Incidencia delictiva
- Pobreza (porcentual)
- Clima
- Restaurantes

### Libros con datos por año para cada provincia:

-Anuario estadístico y geográfico por entidad federativa 2013:
http://inegi.blob.core.windows.net/sistemas/productos/bvinegi/productos/integracion/pais/aepef/2013/aegpef_2013.pdf

-Anuario estadístico y geográfico por entidad federativa 2014:
https://books.google.com.ar/books?id=MmBUDwAAQBAJ&pg=PA2022&lpg=PA2022&dq=turistas+por+entidad+federativa+a%C3%B1o+2012&source=bl&ots=nSf3PUuHHl&sig=ACfU3U3QjotaQpyoTaMQIDri7L5qE50Bcg&hl=en&sa=X&ved=2ahUKEwiPp4DCpv7lAhXCGrkGHTftBzoQ6AEwBnoECAoQAQ#v=onepage&q=superficie%20agricola&f=false

-Anuario estadístico y geográfico por entidad federativa 2015:
http://internet.contenidos.inegi.org.mx/contenidos/Productos/prod_serv/contenidos/espanol/bvinegi/productos/nueva_estruc/AEGPEF_2015/702825077297.pdf

-Anuario estadístico y geográfico por entidad federativa 2015:
http://internet.contenidos.inegi.org.mx/contenidos/Productos/prod_serv/contenidos/espanol/bvinegi/productos/nueva_estruc/AEGPEF_2015/702825077297.pdf

-Anuario estadístico y geográfico por entidad federativa 2016:
http://internet.contenidos.inegi.org.mx/contenidos/Productos/prod_serv/contenidos/espanol/bvinegi/productos/nueva_estruc/AEGPEF_2016/702825087357.pdf

-Anuario estadístico y geográfico por entidad federativa 2018:
http://internet.contenidos.inegi.org.mx/contenidos/Productos/prod_serv/contenidos/espanol/bvinegi/productos/nueva_estruc/AEGPEF_2018/702825107017.pdf

Tienen que buscar los cuadros del mismo tipo en cada libro. Algunos incluyen a 2012.

### Funciones auxiliares

In [6]:
def asignar_delincuencia_por_provincia_fecha(train, df):
    """Recibe el set de datos y un dataset adicional con el index de provincias y columnas con varios años.
    Comprueba para casa en qué año fue publicada y en qué provincia y le asigna el valor de incidencia delictiva
    para ese año y esa provincia, medido cada 100000 habitantes."""
    delincuencia = []
    train["anio"] = train["fecha"].dt.year
    for index, row in train.iterrows():
        provincia = row["provincia"]
        anio = str(row["anio"])
        incidencia = df.loc[provincia][anio]
        delincuencia.append(incidencia)
    train["delincuencia"] = delincuencia
    train.drop(["anio"], axis = 1)
    return train

def asignar_caracteristica_por_fecha(train, df, nombre_columna):
    """Recibe el set de datos y un dataset adicional con el index de provincias y columnas con varios años.
    Comprueba para casa en qué año fue publicada y en qué provincia y le asigna el valor de la columna correspondiente a
    esa provincia y año."""
    caracteristicas = []
    train["anio"] = train["fecha"].dt.year
    for index, row in train.iterrows():
        provincia = row["provincia"]
        anio = str(row["anio"])
        caracteristica = df.loc[provincia][anio]
        caracteristicas.append(caracteristica)
    train[nombre_columna] = caracteristicas
    train.drop(["anio"], axis = 1)
    return train

def rellenar_año_faltante(df):
    df["2015"].fillna(0)
    columna = []
    for index, row in df.iterrows():
        caracteristica_a = row["2014"]
        caracteristica_b = row["2016"]
        if (pd.isna(caracteristica_a) and  not pd.isna(caracteristica_b)):
            columna.append(caracteristica_b)
        elif (not pd.isna(caracteristica_a) and not pd.isna(caracteristica_b)):
            columna.append(int((int(caracteristica_a) + int(caracteristica_b)) / 2))
        else:
            columna.append(caracteristica_a)
    df["2015"] = columna
    return df

### Compruebo los NAN por columna:

In [7]:
train.isna().sum()

titulo                          5387
descripcion                     1619
tipodepropiedad                   46
direccion                      53072
ciudad                           372
provincia                        155
antiguedad                     43555
habitaciones                   22471
garages                        37765
banos                          26221
metroscubiertos                17400
metrostotales                  51467
idzona                         28621
lat                           123488
lng                           123488
fecha                              0
gimnasio                           0
usosmultiples                      0
piscina                            0
escuelascercanas                   0
centroscomercialescercanos         0
precio                             0
dtype: int64

In [8]:
train.dropna(subset=["provincia"], inplace=True)

## INCIDENCIA DELICTIVA

In [9]:
incidencia_delictiva = pd.read_csv("data/Tasa_de_incidencia_delictiva_por_entidad_federativa_de_ocurrencia_por_cada_cien_mil_habitantes.csv")

In [10]:
incidencia_delictiva.head()

,Entidad,2010 /1,2011 /2,2012 /3,2013 /4,2014,2015,2016,2017,2018
0,Estados Unidos Mexicanos,30535,29200,35139,41563,41655,35497,37017,39369,37807
1,Aguascalientes,56089,25511,32368,24711,39453,35457,41254,39912,36500
2,Baja California,31791,29446,39297,57066,56632,32758,51286,43921,42725
3,Baja California Sur,25779,28884,31049,23747,34700,25577,29939,25690,28377
4,Campeche,20922,21704,29097,30597,29306,22114,28892,28283,26466


### Compruebo las fechas máximas y mínimas de train

In [11]:
train["fecha"].max()

Timestamp('2016-12-31 00:00:00')

In [12]:
train["fecha"].min()

Timestamp('2012-01-01 00:00:00')

Me quedo sólo con los años que están en train

In [13]:
incidencia_delictiva = incidencia_delictiva[["Entidad", "2012 /3", "2013 /4", "2014", "2015", "2016"]]

### Compruebo las provincias:

In [14]:
incidencia_delictiva.Entidad.unique()

array(['Estados Unidos Mexicanos', 'Aguascalientes', 'Baja California',
       'Baja California Sur', 'Campeche', 'Coahuila de Zaragoza',
       'Colima', 'Chiapas', 'Chihuahua', 'Ciudad de México', 'Durango',
       'Guanajuato', 'Guerrero', 'Hidalgo', 'Jalisco', 'México',
       'Michoacán de Ocampo', 'Morelos', 'Nayarit', 'Nuevo León',
       'Oaxaca', 'Puebla', 'Querétaro', 'Quintana Roo', 'San Luis Potosí',
       'Sinaloa', 'Sonora', 'Tabasco', 'Tamaulipas', 'Tlaxcala',
       'Veracruz de Ignacio de la Llave', 'Yucatán', 'Zacatecas'],
      dtype=object)

Cambio los nombres para que coincidan con train

In [15]:
incidencia_delictiva.replace({"Baja California": "Baja California Norte",
                              "Coahuila de Zaragoza": "Coahuila",
                              "Michoacán de Ocampo": "Michoacán",
                              "Veracruz de Ignacio de la Llave": "Veracruz",
                              "México": "Edo. de México",
                              "Ciudad de México": "Distrito Federal",
                              "San Luis Potosí": "San luis Potosí"}, inplace = True)

In [16]:
incidencia_delictiva.columns = ["provincia", "2012", "2013", "2014", "2015", "2016"]

In [17]:
incidencia_delictiva.head()

,provincia,2012,2013,2014,2015,2016
0,Estados Unidos Mexicanos,35139,41563,41655,35497,37017
1,Aguascalientes,32368,24711,39453,35457,41254
2,Baja California Norte,39297,57066,56632,32758,51286
3,Baja California Sur,31049,23747,34700,25577,29939
4,Campeche,29097,30597,29306,22114,28892


In [18]:
incidencia_delictiva = incidencia_delictiva.set_index("provincia")
incidencia_delictiva.head()

,2012,2013,2014,2015,2016
provincia,,,,,
Estados Unidos Mexicanos,35139,41563,41655,35497,37017
Aguascalientes,32368,24711,39453,35457,41254
Baja California Norte,39297,57066,56632,32758,51286
Baja California Sur,31049,23747,34700,25577,29939
Campeche,29097,30597,29306,22114,28892


In [19]:
#train["anio"] = train["fecha"].dt.year

In [20]:
#train = asignar_delincuencia_por_provincia_fecha(train, incidencia_delictiva)

In [21]:
train.head()

,titulo,descripcion,tipodepropiedad,direccion,ciudad,provincia,antiguedad,habitaciones,garages,banos,...,idzona,lat,lng,fecha,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos,precio
id,,,,,,,,,,,,,,,,,,,,,
254099,depto. tipo a-402,"depto. interior de 80.15m2, consta de sala com...",Apartamento,Avenida Division del Norte 2005,Benito Juárez,Distrito Federal,NaN,2.0,1.0,2.0,...,23533.0,NaN,NaN,2015-08-23,0.0,0.0,0.0,0.0,0.0,2273000.0
53461,condominio horizontal en venta,"<p>entre sonora y guerrero, atr&aacute;s del h...",Casa en condominio,AV. MEXICO,La Magdalena Contreras,Distrito Federal,10.0,3.0,2.0,2.0,...,24514.0,19.310205,-99.227655,2013-06-28,0.0,0.0,0.0,1.0,1.0,3600000.0
247984,casa en venta urbi 3 recamaras tonala,descripcion \nla mejor ubicacion residencial e...,Casa,Urbi Tonala,Tonalá,Jalisco,5.0,3.0,2.0,2.0,...,48551.0,NaN,NaN,2015-10-17,0.0,0.0,0.0,0.0,0.0,1200000.0
209067,casa sola en toluca zinacantepec con credito i...,casa en privada con caseta de vigilancia casas...,Casa,IGNACIO MANUEL ALTAMIRANO 128,Zinacantepec,Edo. de México,1.0,2.0,1.0,1.0,...,53666.0,19.301890,-99.688015,2012-03-09,0.0,0.0,0.0,1.0,1.0,650000.0
185997,paseos del sol,bonito departamento en excelentes condiciones ...,Apartamento,PASEOS DEL SOL,Zapopan,Jalisco,10.0,2.0,1.0,1.0,...,47835.0,NaN,NaN,2016-06-07,0.0,0.0,0.0,0.0,0.0,1150000.0


## POBREZA

In [22]:
pobreza = pd.read_csv("data/pobreza_porcentual.csv")

In [23]:
pobreza.head()

,provincia,2008,2010,2012,2014,2016,2018
0,Aguascalientes,37.6,38.1,37.8,34.8,28.2,26.2
1,Baja California,26.0,31.5,30.2,28.6,22.2,23.3
2,Baja California Sur,21.4,31.0,30.1,30.3,22.1,18.1
3,Campeche,45.9,50.5,44.7,43.6,43.8,46.2
4,Coahuila,32.7,27.8,27.9,30.2,24.8,22.5


Faltan datos de los impares. Saco el promedio entre los años contiguos para obtener su valor.

In [24]:
pobreza["2013"] = (pobreza["2012"] + pobreza["2014"]) / 2
pobreza["2015"] = (pobreza["2014"] + pobreza["2016"]) / 2

In [25]:
pobreza.head()

,provincia,2008,2010,2012,2014,2016,2018,2013,2015
0,Aguascalientes,37.6,38.1,37.8,34.8,28.2,26.2,36.30,31.5
1,Baja California,26.0,31.5,30.2,28.6,22.2,23.3,29.40,25.4
2,Baja California Sur,21.4,31.0,30.1,30.3,22.1,18.1,30.20,26.2
3,Campeche,45.9,50.5,44.7,43.6,43.8,46.2,44.15,43.7
4,Coahuila,32.7,27.8,27.9,30.2,24.8,22.5,29.05,27.5


### Compruebo las provincias:

In [26]:
pobreza_provincias = pobreza["provincia"].unique()
pobreza_provincias.sort()
pobreza_provincias

array(['Aguascalientes', 'Baja California', 'Baja California Sur',
       'Campeche', 'Chiapas', 'Chihuahua', 'Ciudad de México', 'Coahuila',
       'Colima', 'Durango', 'Guanajuato', 'Guerrero', 'Hidalgo',
       'Jalisco', 'Michoacán', 'Morelos', 'México', 'Nayarit',
       'Nuevo León', 'Oaxaca', 'Puebla', 'Querétaro', 'Quintana Roo',
       'San Luis Potosí', 'Sinaloa', 'Sonora', 'Tabasco', 'Tamaulipas',
       'Tlaxcala', 'Veracruz', 'Yucatán', 'Zacatecas'], dtype=object)

In [27]:
train_provincias = train["provincia"].unique()
train_provincias.sort()
train_provincias

array(['Aguascalientes', 'Baja California Norte', 'Baja California Sur',
       'Campeche', 'Chiapas', 'Chihuahua', 'Coahuila', 'Colima',
       'Distrito Federal', 'Durango', 'Edo. de México', 'Guanajuato',
       'Guerrero', 'Hidalgo', 'Jalisco', 'Michoacán', 'Morelos',
       'Nayarit', 'Nuevo León', 'Oaxaca', 'Puebla', 'Querétaro',
       'Quintana Roo', 'San luis Potosí', 'Sinaloa', 'Sonora', 'Tabasco',
       'Tamaulipas', 'Tlaxcala', 'Veracruz', 'Yucatán', 'Zacatecas'],
      dtype=object)

In [28]:
pobreza.replace({"Baja California": "Baja California Norte",
                 "México": "Edo. de México",
                 "Ciudad de México": "Distrito Federal",
                 "San Luis Potosí": "San luis Potosí"}, inplace = True)

In [29]:
pobreza = pobreza.set_index("provincia")

In [30]:
#train = asignar_caracteristica_por_fecha(train, pobreza, "pobreza_porcentual")

In [31]:
pobreza

,2008,2010,2012,2014,2016,2018,2013,2015
provincia,,,,,,,,
Aguascalientes,37.6,38.1,37.8,34.8,28.2,26.2,36.30,31.50
Baja California Norte,26.0,31.5,30.2,28.6,22.2,23.3,29.40,25.40
Baja California Sur,21.4,31.0,30.1,30.3,22.1,18.1,30.20,26.20
Campeche,45.9,50.5,44.7,43.6,43.8,46.2,44.15,43.70
Coahuila,32.7,27.8,27.9,30.2,24.8,22.5,29.05,27.50
Colima,27.4,34.7,34.4,34.3,33.6,30.9,34.35,33.95
Chiapas,77.0,78.5,74.7,76.2,77.1,76.4,75.45,76.65
Chihuahua,32.1,38.8,35.3,34.4,30.6,26.3,34.85,32.50
Distrito Federal,27.6,28.5,28.9,28.4,27.6,30.6,28.65,28.00


In [32]:
train

,titulo,descripcion,tipodepropiedad,direccion,ciudad,provincia,antiguedad,habitaciones,garages,banos,...,idzona,lat,lng,fecha,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos,precio
id,,,,,,,,,,,,,,,,,,,,,
254099,depto. tipo a-402,"depto. interior de 80.15m2, consta de sala com...",Apartamento,Avenida Division del Norte 2005,Benito Juárez,Distrito Federal,NaN,2.0,1.0,2.0,...,23533.0,NaN,NaN,2015-08-23,0.0,0.0,0.0,0.0,0.0,2273000.0
53461,condominio horizontal en venta,"<p>entre sonora y guerrero, atr&aacute;s del h...",Casa en condominio,AV. MEXICO,La Magdalena Contreras,Distrito Federal,10.0,3.0,2.0,2.0,...,24514.0,19.310205,-99.227655,2013-06-28,0.0,0.0,0.0,1.0,1.0,3600000.0
247984,casa en venta urbi 3 recamaras tonala,descripcion \nla mejor ubicacion residencial e...,Casa,Urbi Tonala,Tonalá,Jalisco,5.0,3.0,2.0,2.0,...,48551.0,NaN,NaN,2015-10-17,0.0,0.0,0.0,0.0,0.0,1200000.0
209067,casa sola en toluca zinacantepec con credito i...,casa en privada con caseta de vigilancia casas...,Casa,IGNACIO MANUEL ALTAMIRANO 128,Zinacantepec,Edo. de México,1.0,2.0,1.0,1.0,...,53666.0,19.301890,-99.688015,2012-03-09,0.0,0.0,0.0,1.0,1.0,650000.0
185997,paseos del sol,bonito departamento en excelentes condiciones ...,Apartamento,PASEOS DEL SOL,Zapopan,Jalisco,10.0,2.0,1.0,1.0,...,47835.0,NaN,NaN,2016-06-07,0.0,0.0,0.0,0.0,0.0,1150000.0
126147,departamento en venta taxqueña,"amplio departamento, estancia de sala y comedo...",Apartamento,Condominio Tlalpan 2B,Coyoacán,Distrito Federal,5.0,2.0,1.0,1.0,...,23650.0,19.300612,-99.148475,2014-03-18,0.0,0.0,0.0,0.0,1.0,1100000.0
139233,de oportunidad casa en san lorenzo,"ubicada en esquina, pertenece san lorenzo agen...",Casa,NaN,Oaxaca de Juárez,Oaxaca,NaN,3.0,1.0,2.0,...,73510.0,17.143504,-96.803504,2016-02-23,0.0,0.0,0.0,0.0,0.0,1150000.0
5013,casa emilia en venta en selvamar playa del carmen,casa emilia en venta playa del carmenfracciona...,Casa,condominio el trebol,Playa del Carmen,Quintana Roo,2.0,4.0,2.0,3.0,...,130510.0,20.672612,-87.037968,2016-10-20,0.0,0.0,0.0,0.0,0.0,4200000.0
44962,pre- venta preciosos depas 2 recamaras con sub...,<p>pre-venta de preciosos departamento ecologi...,Apartamento,BUENAVISTA DEPTOS CON SUBSIDIO,Villa de Alvarez,Colima,1.0,2.0,1.0,1.0,...,9010.0,NaN,NaN,2014-01-06,0.0,0.0,0.0,1.0,1.0,310000.0


## Clima

In [33]:
clima = pd.read_csv("data/clima.csv")

In [34]:
clima

,provincia,Intervalo de temperatura media annual (minimo),Intervalo de temperatura media annual (máximo),Seco,Templado,Cálido,Frío
0,Aguascalientes,12,22,1,1,0,0
1,Baja California Norte,6,24,1,1,0,0
2,Baja California Sur,14,26,1,1,0,0
3,Campeche,24,26,0,0,1,0
4,Coahuila,8,22,1,1,0,0
5,Colima,10,28,1,1,1,0
6,Chiapas,10,30,0,1,1,0
7,Chihuahua,8,26,1,1,1,0
8,Distrito Federal,6,18,1,1,0,0
9,Durango,8,26,1,1,1,0


### Compruebo las provincias

In [35]:
clima_provincias = clima["provincia"].unique()
clima_provincias.sort()
clima_provincias

array(['Aguascalientes', 'Baja California Norte', 'Baja California Sur',
       'Campeche', 'Chiapas', 'Chihuahua', 'Coahuila ', 'Colima',
       'Distrito Federal', 'Durango', 'Guanajuato', 'Guerrero', 'Hidalgo',
       'Jalisco', 'Michoacán', 'Morelos', 'México', 'Nayarit',
       'Nuevo León', 'Oaxaca', 'Puebla', 'Querétaro', 'Quintana Roo',
       'San luis Potosí', 'Sinaloa', 'Sonora', 'Tabasco', 'Tamaulipas',
       'Tlaxcala', 'Veracruz', 'Yucatán', 'Zacatecas'], dtype=object)

In [36]:
train_provincias

array(['Aguascalientes', 'Baja California Norte', 'Baja California Sur',
       'Campeche', 'Chiapas', 'Chihuahua', 'Coahuila', 'Colima',
       'Distrito Federal', 'Durango', 'Edo. de México', 'Guanajuato',
       'Guerrero', 'Hidalgo', 'Jalisco', 'Michoacán', 'Morelos',
       'Nayarit', 'Nuevo León', 'Oaxaca', 'Puebla', 'Querétaro',
       'Quintana Roo', 'San luis Potosí', 'Sinaloa', 'Sonora', 'Tabasco',
       'Tamaulipas', 'Tlaxcala', 'Veracruz', 'Yucatán', 'Zacatecas'],
      dtype=object)

In [37]:
clima.replace({"México": "Edo. de México"}, inplace = True)

In [38]:
#train = pd.merge(train, clima, on = "provincia", how = "left")

In [39]:
train.head()

,titulo,descripcion,tipodepropiedad,direccion,ciudad,provincia,antiguedad,habitaciones,garages,banos,...,idzona,lat,lng,fecha,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos,precio
id,,,,,,,,,,,,,,,,,,,,,
254099,depto. tipo a-402,"depto. interior de 80.15m2, consta de sala com...",Apartamento,Avenida Division del Norte 2005,Benito Juárez,Distrito Federal,NaN,2.0,1.0,2.0,...,23533.0,NaN,NaN,2015-08-23,0.0,0.0,0.0,0.0,0.0,2273000.0
53461,condominio horizontal en venta,"<p>entre sonora y guerrero, atr&aacute;s del h...",Casa en condominio,AV. MEXICO,La Magdalena Contreras,Distrito Federal,10.0,3.0,2.0,2.0,...,24514.0,19.310205,-99.227655,2013-06-28,0.0,0.0,0.0,1.0,1.0,3600000.0
247984,casa en venta urbi 3 recamaras tonala,descripcion \nla mejor ubicacion residencial e...,Casa,Urbi Tonala,Tonalá,Jalisco,5.0,3.0,2.0,2.0,...,48551.0,NaN,NaN,2015-10-17,0.0,0.0,0.0,0.0,0.0,1200000.0
209067,casa sola en toluca zinacantepec con credito i...,casa en privada con caseta de vigilancia casas...,Casa,IGNACIO MANUEL ALTAMIRANO 128,Zinacantepec,Edo. de México,1.0,2.0,1.0,1.0,...,53666.0,19.301890,-99.688015,2012-03-09,0.0,0.0,0.0,1.0,1.0,650000.0
185997,paseos del sol,bonito departamento en excelentes condiciones ...,Apartamento,PASEOS DEL SOL,Zapopan,Jalisco,10.0,2.0,1.0,1.0,...,47835.0,NaN,NaN,2016-06-07,0.0,0.0,0.0,0.0,0.0,1150000.0


## TURISMO (restaurantes, discotecas, etc)

In [40]:
turismo_2012 = pd.read_csv("data/servicio_turistico_2012.csv")

In [41]:
turismo_2013 = pd.read_csv("data/servicio_turistico_2013.csv")

In [42]:
turismo_2014 = pd.read_csv("data/servicio_turistico_2014.csv")

In [43]:
turismo_2016 = pd.read_csv("data/servicio_turistico_2016.csv")

In [44]:
turismo_2012["provincia"].unique()

array(['Aguascalientes', 'Baja California Norte', 'Baja California Sur',
       'Campeche', 'Coahuila', 'Colima', 'Chiapas', 'Chihuahua',
       'Distrito Federal', 'Durango', 'Guanajuato', 'Guerrero', 'Hidalgo',
       'Jalisco', 'Edo. de México', 'Michoacán', 'Morelos', 'Nayarit',
       'Nuevo León', 'Oaxaca', 'Puebla', 'Querétaro', 'Quintana Roo',
       'San luis Potosí', 'Sinaloa', 'Sonora', 'Tabasco', 'Tamaulipas',
       'Tlaxcala', 'Veracruz', 'Yucatán', 'Zacatecas'], dtype=object)

In [45]:
turismo_2013["provincia"].unique()

array(['Aguascalientes', 'Baja California Norte', 'Baja California Sur',
       'Campeche', 'Coahuila', 'Colima', 'Chiapas', 'Chihuahua',
       'Distrito Federal', 'Durango', 'Guanajuato', 'Guerrero', 'Hidalgo',
       'Jalisco', 'Edo. de México', 'Michoacán', 'Morelos', 'Nayarit',
       'Nuevo León', 'Oaxaca', 'Puebla', 'Querétaro', 'Quintana Roo',
       'San luis Potosí', 'Sinaloa', 'Sonora', 'Tabasco', 'Tamaulipas',
       'Tlaxcala', 'Veracruz', 'Yucatán', 'Zacatecas'], dtype=object)

In [46]:
turismo_2014["provincia"].unique()

array(['Aguascalientes', 'Baja California Norte', 'Baja California Sur',
       'Campeche', 'Coahuila', 'Colima', 'Chiapas', 'Chihuahua',
       'Distrito Federal', 'Durango', 'Guanajuato', 'Guerrero', 'Hidalgo',
       'Jalisco', 'Edo. de México', 'Michoacán', 'Morelos', 'Nayarit',
       'Nuevo León', 'Oaxaca', 'Puebla', 'Querétaro', 'Quintana Roo',
       'San luis Potosí', 'Sinaloa', 'Sonora', 'Tabasco', 'Tamaulipas',
       'Tlaxcala', 'Veracruz', 'Yucatán', 'Zacatecas'], dtype=object)

In [47]:
turismo_2016["provincia"].unique()

array(['Aguascalientes', 'Baja California Norte', 'Baja California Sur',
       'Campeche', 'Coahuila', 'Colima', 'Chiapas', 'Chihuahua',
       'Distrito Federal\t', 'Durango', 'Guanajuato', 'Guerrero',
       'Hidalgo', 'Jalisco', 'Edo. de México', 'Michocán', 'Morelos',
       'Nayarit', 'Nuevo León', 'Oaxaca', 'Puebla', 'Querétaro',
       'Quintana Roo', 'San luis Potosí', 'Sinaloa', 'Sonora', 'Tabasco',
       'Tamaulipas', 'Tlaxcala', 'Veracruz', 'Yucatán', 'Zacatecas'],
      dtype=object)

In [48]:
turismo_2016.replace({"Distrito Federal\t": "Distrito Federal"}, inplace = True)

In [49]:
turismo_2012.head()

,provincia,total,restaurantes,cafeterias,discotecas y centros nocturnos,bares,otros
0,Aguascalientes,359,247,51,16,45,ND
1,Baja California Norte,525,361,46,35,68,15
2,Baja California Sur,388,314,20,25,29,ND
3,Campeche,807,613,59,34,101,ND
4,Coahuila,884,715,40,31,98,ND


### Restaurantes

In [50]:
restaurantes = pd.DataFrame(columns=["provincia", "2012", "2013", "2014", "2015", "2016"])

In [51]:
restaurantes["provincia"] = turismo_2012["provincia"]
restaurantes["2012"] = turismo_2012["restaurantes"]
restaurantes["2013"] = turismo_2013["restaurantes"]
restaurantes["2014"] = turismo_2014["restaurantes"]
restaurantes["2016"] = turismo_2016["restaurantes"]
restaurantes.replace({"ND": np.nan}, inplace = True)

In [52]:
restaurantes = rellenar_año_faltante(restaurantes)

In [53]:
restaurantes = restaurantes.T.fillna(restaurantes.mean(axis=1)).T

In [54]:
restaurantes = restaurantes.set_index("provincia")

In [55]:
#train = asignar_caracteristica_por_fecha(train, restaurantes, "restaurantes")

In [56]:
restaurantes

,2012,2013,2014,2015,2016
provincia,,,,,
Aguascalientes,247,265,303,360,417
Baja California Norte,361,361,361,1718,1718
Baja California Sur,314,328,310,542,775
Campeche,613,607,607,610,614
Coahuila,715,715,2210,2234,2259
Colima,302,303,298,314,330
Chiapas,1265,1290,1293,1297,1301
Chihuahua,2258,2467,2631,2631,2258
Distrito Federal,1116,1074,967,943,920


## PBI

In [57]:
pbi_2012 = pd.read_csv("data/pbi_2012.csv")
pbi_2013 = pd.read_csv("data/pbi_2013.csv")
pbi_2014 = pd.read_csv("data/pbi_2014.csv")
pbi_2015 = pd.read_csv("data/pbi_2015.csv")
pbi_2016 = pd.read_csv("data/pbi_2016.csv")

In [58]:
pbis = [pbi_2012, pbi_2013, pbi_2014, pbi_2015, pbi_2016]

In [59]:
def crear_dataframe(dataframes, columna):
    columnas = ["provincia", "2012", "2013", "2014", "2015", "2016"]
    df = pd.DataFrame(columns=columnas)
    df[columnas[0]] = dataframes[0]["provincia"] 
    for i in range (0, len(dataframes)):
        df[columnas[i + 1]] =  dataframes[i][columna]
    return df

In [60]:
pbi_campo = crear_dataframe(pbis, "pbi_campo")
pbi_campo = pbi_campo.set_index("provincia")

pbi_mineria = crear_dataframe(pbis, "pbi_minería")
pbi_mineria = pbi_mineria.set_index("provincia")

pbi_energia_agua_gas = crear_dataframe(pbis, "pbi_energia_agua_gas")
pbi_energia_agua_gas = pbi_energia_agua_gas.set_index("provincia")

pbi_construccion = crear_dataframe(pbis, "pbi_construccion")
pbi_construccion = pbi_construccion.set_index("provincia")

pbi_industrias_manufactureras = crear_dataframe(pbis, "pbi_industrias_manufactureras")
pbi_industrias_manufactureras = pbi_industrias_manufactureras.set_index("provincia")

pbi_comercio = crear_dataframe(pbis, "pbi_comercio")
pbi_comercio = pbi_comercio.set_index("provincia")

In [61]:
#train = asignar_caracteristica_por_fecha(train, pbi_campo, "pbi_campo")



In [62]:
train

,titulo,descripcion,tipodepropiedad,direccion,ciudad,provincia,antiguedad,habitaciones,garages,banos,...,idzona,lat,lng,fecha,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos,precio
id,,,,,,,,,,,,,,,,,,,,,
254099,depto. tipo a-402,"depto. interior de 80.15m2, consta de sala com...",Apartamento,Avenida Division del Norte 2005,Benito Juárez,Distrito Federal,NaN,2.0,1.0,2.0,...,23533.0,NaN,NaN,2015-08-23,0.0,0.0,0.0,0.0,0.0,2273000.0
53461,condominio horizontal en venta,"<p>entre sonora y guerrero, atr&aacute;s del h...",Casa en condominio,AV. MEXICO,La Magdalena Contreras,Distrito Federal,10.0,3.0,2.0,2.0,...,24514.0,19.310205,-99.227655,2013-06-28,0.0,0.0,0.0,1.0,1.0,3600000.0
247984,casa en venta urbi 3 recamaras tonala,descripcion \nla mejor ubicacion residencial e...,Casa,Urbi Tonala,Tonalá,Jalisco,5.0,3.0,2.0,2.0,...,48551.0,NaN,NaN,2015-10-17,0.0,0.0,0.0,0.0,0.0,1200000.0
209067,casa sola en toluca zinacantepec con credito i...,casa en privada con caseta de vigilancia casas...,Casa,IGNACIO MANUEL ALTAMIRANO 128,Zinacantepec,Edo. de México,1.0,2.0,1.0,1.0,...,53666.0,19.301890,-99.688015,2012-03-09,0.0,0.0,0.0,1.0,1.0,650000.0
185997,paseos del sol,bonito departamento en excelentes condiciones ...,Apartamento,PASEOS DEL SOL,Zapopan,Jalisco,10.0,2.0,1.0,1.0,...,47835.0,NaN,NaN,2016-06-07,0.0,0.0,0.0,0.0,0.0,1150000.0
126147,departamento en venta taxqueña,"amplio departamento, estancia de sala y comedo...",Apartamento,Condominio Tlalpan 2B,Coyoacán,Distrito Federal,5.0,2.0,1.0,1.0,...,23650.0,19.300612,-99.148475,2014-03-18,0.0,0.0,0.0,0.0,1.0,1100000.0
139233,de oportunidad casa en san lorenzo,"ubicada en esquina, pertenece san lorenzo agen...",Casa,NaN,Oaxaca de Juárez,Oaxaca,NaN,3.0,1.0,2.0,...,73510.0,17.143504,-96.803504,2016-02-23,0.0,0.0,0.0,0.0,0.0,1150000.0
5013,casa emilia en venta en selvamar playa del carmen,casa emilia en venta playa del carmenfracciona...,Casa,condominio el trebol,Playa del Carmen,Quintana Roo,2.0,4.0,2.0,3.0,...,130510.0,20.672612,-87.037968,2016-10-20,0.0,0.0,0.0,0.0,0.0,4200000.0
44962,pre- venta preciosos depas 2 recamaras con sub...,<p>pre-venta de preciosos departamento ecologi...,Apartamento,BUENAVISTA DEPTOS CON SUBSIDIO,Villa de Alvarez,Colima,1.0,2.0,1.0,1.0,...,9010.0,NaN,NaN,2014-01-06,0.0,0.0,0.0,1.0,1.0,310000.0


## Unión de los dataframes con train y test

### Train

In [63]:
#train = asignar_caracteristica_por_fecha(train, incidencia_delictiva, "incidencia_delictiva")

In [64]:
#train = asignar_caracteristica_por_fecha(train, pobreza, "pobreza_porcentual")

In [65]:
#train = pd.merge(train, clima, on = "provincia", how = "left")

In [66]:
#train = asignar_caracteristica_por_fecha(train, restaurantes, "restaurantes")

In [67]:
#train = asignar_caracteristica_por_fecha(train, pbi_campo, "pbi_campo")
#train = asignar_caracteristica_por_fecha(train, pbi_mineria, "pbi_mineria")
#train = asignar_caracteristica_por_fecha(train, pbi_energia_agua_gas, "pbi_energia_agua_gas")
#train = asignar_caracteristica_por_fecha(train, pbi_construccion, "pbi_construccion")
#train = asignar_caracteristica_por_fecha(train, pbi_industrias_manufactureras, "pbi_industrias_manufactureras")
#train = asignar_caracteristica_por_fecha(train, pbi_comercio, "pbi_comercio")

### Prueba de agregar features

In [70]:
test = test.dropna()

In [71]:
test = feature.agregar_feature_incidencia_delictiva(test)

In [72]:
test = feature.agregar_feature_pobreza_porcentual(test)

In [73]:
test = feature.agregar_feature_clima(test)

In [75]:
test = feature.agregar_feature_restaurantes(test)

In [76]:
test = feature.agregar_feature_pbi(test)

In [80]:
test.head()

,titulo,descripcion,tipodepropiedad,direccion,ciudad,provincia,antiguedad,habitaciones,garages,banos,...,Templado,Cálido,Frío,restaurantes,pbi_campo,pbi_mineria,pbi_energia_agua_gas,pbi_construccion,pbi_industrias_manufactureras,pbi_comercio
0,departamento nuevo delegación coyoacán de 87 m...,"departamento nuevo de 87.06 m2, 1 cajón de est...",Apartamento,"Pueblo de los Reyes, Coyoacán, Mexico D.F.",Coyoacán,Distrito Federal,0.0,2.0,1.0,2.0,...,1.0,0.0,0.0,943,994,120,13989,45092,172748,280309
1,bonita casa sola equipada de dos niveles en lo...,"<p>casa sola, bonita de dos rec&aacute;maras u...",Casa,CEDROS,Tultitlán,Edo. de México,10.0,2.0,1.0,1.0,...,1.0,1.0,1.0,1789,17152,2282,20212,91074,281887,219813
2,hermosa casa con alberca y acceso a la playa,"hermosa residencia, con alberca y acceso a la ...",Casa,Americas 2,Veracruz,Veracruz,2.0,3.0,2.0,2.0,...,1.0,1.0,1.0,5878,48334,27325,21857,79100,148343,94350
3,casa en sm 30 cancun,descripción: \nplanta baja \n2 recamaras \n1 b...,Casa,YALAHAN,Cancún,Quintana Roo,20.0,3.0,1.0,2.0,...,0.0,1.0,1.0,1378,2027,1621,2304,20081,4106,16616
4,"cad laguna, 3 recamaras, condominio en fairway...","cad cielo 301, 3 recamaras, condominio en fair...",Apartamento,"Fraccionamiento Copacabana, Acapulco, Guerrero",Acapulco de Juárez,Guerrero,0.0,3.0,2.0,3.0,...,1.0,1.0,0.0,699,15211,9831,9534,23312,6973,25674
